# Wire Bounding Box Detection - YOLOv11 Training
Fine-tune YOLOv11s for wire detection using Apple Silicon (MPS)

In [ ]:
# !pip install ultralytics

In [ ]:
import os
from pathlib import Path
from ultralytics import YOLO
import torch

print(f'PyTorch: {torch.__version__}')
print(f'MPS available: {torch.backends.mps.is_available()}')

## Configuration

In [ ]:
# Dataset path (YOLO format with data.yaml)
DATA_YAML = '<path_to_data>'

# Output directory for trained model
OUTPUT_DIR = './checkpoints/yolo11s_wires'

# Model variant: yolo11n (nano), yolo11s (small), yolo11m (medium)
MODEL = 'yolo11s.pt'  # small - good balance for wire detection

# Training parameters
EPOCHS = 50
BATCH_SIZE = 16
IMG_SIZE = 640  # input resolution
PATIENCE = 10   # early stopping patience

# Device
if torch.backends.mps.is_available():
    DEVICE = 'mps'
elif torch.cuda.is_available():
    DEVICE = 0  # cuda device id
else:
    DEVICE = 'cpu'

print(f'Device: {DEVICE}')
print(f'Model: {MODEL}')
print(f'Epochs: {EPOCHS}')
print(f'Batch size: {BATCH_SIZE}')

## Load Model

In [ ]:
# Load pretrained YOLOv11
model = YOLO(MODEL)
print(f'Loaded {MODEL}')

## Training

In [ ]:
# Train the model
results = model.train(
    data=DATA_YAML,
    epochs=EPOCHS,
    batch=BATCH_SIZE,
    imgsz=IMG_SIZE,
    device=DEVICE,
    patience=PATIENCE,
    project=OUTPUT_DIR,
    name='train',
    exist_ok=True,
    pretrained=True,
    optimizer='AdamW',
    lr0=0.001,
    lrf=0.01,
    warmup_epochs=3,
    close_mosaic=10,
    plots=True,
    save=True,
    val=True,
)

print('\nTraining complete!')

## Evaluate on Validation Set

In [ ]:
# Load best checkpoint and validate
best_model = YOLO(f'{OUTPUT_DIR}/train/weights/best.pt')
val_results = best_model.val(data=DATA_YAML)

print(f"\nmAP50: {val_results.box.map50:.4f}")
print(f"mAP50-95: {val_results.box.map:.4f}")

## Video Inference

In [ ]:
import cv2
import time

def process_video(model, input_path, output_path, conf=0.5, display=False):
    """Process video with trained model."""
    cap = cv2.VideoCapture(input_path)
    
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    print(f'Processing: {width}x{height} @ {fps:.1f} fps ({total_frames} frames)')
    
    frame_count = 0
    total_time = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Predict
        start = time.time()
        results = model.predict(frame, conf=conf, verbose=False)
        total_time += time.time() - start
        
        # Draw results
        annotated = results[0].plot()
        out.write(annotated)
        frame_count += 1
        
        if frame_count % 100 == 0:
            avg_fps = frame_count / total_time
            print(f'Progress: {frame_count}/{total_frames} ({avg_fps:.1f} FPS)')
        
        if display:
            cv2.imshow('Output', annotated)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
    cap.release()
    out.release()
    if display:
        cv2.destroyAllWindows()
    
    avg_fps = frame_count / total_time
    print(f'\nDone. Saved to {output_path}')
    print(f'Average inference: {avg_fps:.1f} FPS ({1000/avg_fps:.1f}ms per frame)')
    return frame_count

In [ ]:
# Process a video (update paths)
# process_video(
#     best_model,
#     input_path='/path/to/input.mp4',
#     output_path='/path/to/output.mp4',
#     conf=0.5,
#     display=True
# )